In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/bus/tour-times-chennai"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages(driver):
    for page in range(1, 3):  # There are 2 pages
        try:
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Initialize the driver outside the loop
driver = initialize_driver()

# Scrape routes and details from all pages
scrape_all_pages(driver)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('chennais_bus_details.csv', index=False)

# Close the driver after scraping is done
if driver:
    driver.quit()


In [5]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Chennai,https://www.redbus.in/bus-tickets/bangalore-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:10,06h 55m,06:05,4.5,INR 1209,28 Seats available
1,Bangalore to Chennai,https://www.redbus.in/bus-tickets/bangalore-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:20,07h 30m,06:50,4.4,INR 929,27 Seats available
2,Bangalore to Chennai,https://www.redbus.in/bus-tickets/bangalore-to...,NueGo,Electric A/C Seater/Sleeper (2+1),22:00,07h 05m,05:05,4.1,1309,15 Seats available
3,Bangalore to Chennai,https://www.redbus.in/bus-tickets/bangalore-to...,AdDLT BUS,A/C Sleeper (2+1),23:45,06h 30m,06:15,4.0,INR 1620,19 Seats available
4,Bangalore to Chennai,https://www.redbus.in/bus-tickets/bangalore-to...,SPS Travels,A/C Sleeper (2+1),21:55,06h 40m,04:35,4.4,INR 1629,17 Seats available
...,...,...,...,...,...,...,...,...,...,...
1187,Chennai to Thanjavur,https://www.redbus.in/bus-tickets/chennai-to-t...,Yolo Bus,A/C Sleeper (2+1),22:30,06h 25m,04:55,4.5,INR 1269,20 Seats available
1188,Chennai to Thanjavur,https://www.redbus.in/bus-tickets/chennai-to-t...,Yolo Bus,,21:40,07h 19m,04:59,,,
1189,Chennai to Thanjavur,https://www.redbus.in/bus-tickets/chennai-to-t...,,,,,,,,
1190,Chennai to Thanjavur,https://www.redbus.in/bus-tickets/chennai-to-t...,,,,,,,,


In [11]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Chennai to Bangalore,https://www.redbus.in/bus-tickets/chennai-to-b...,KMRL Kalaimakal,A/C Seater / Sleeper (2+1),22:45,06h 45m,05:30,4.8,INR 840,27 Seats available
1,Chennai to Bangalore,https://www.redbus.in/bus-tickets/chennai-to-b...,Asian Xpress,Bharat Benz A/C Semi Sleeper (2+2),23:10,06h 25m,05:35,4.8,INR 695,21 Seats available
2,Chennai to Bangalore,https://www.redbus.in/bus-tickets/chennai-to-b...,Jai Sai Baba Travels,A/C Sleeper (2+1),14:30,07h 25m,21:55,4.8,849,22 Seats available
3,Chennai to Bangalore,https://www.redbus.in/bus-tickets/chennai-to-b...,AdIntrCity SmartBus,AC Sleeper (2+1),21:15,07h 30m,04:45,4.5,INR 1109,27 Seats available
4,Chennai to Bangalore,https://www.redbus.in/bus-tickets/chennai-to-b...,KBS Travels,A/C Sleeper (2+1),20:35,06h 25m,03:00,4.6,925,24 Seats available
...,...,...,...,...,...,...,...,...,...,...
1008,Hyderabad to Chennai,https://www.redbus.in/bus-tickets/hyderabad-to...,,,,,,,,
1009,Hyderabad to Chennai,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,AC Sleeper (2+1),19:15,10h 30m,05:45,4.6,INR 1369,22 Seats available
1010,Hyderabad to Chennai,https://www.redbus.in/bus-tickets/hyderabad-to...,Ramana Tours And Travels,A/C Sleeper (2+1),17:45,14h 10m,07:55,4.5,INR 1400,10 Seats available
1011,Hyderabad to Chennai,https://www.redbus.in/bus-tickets/hyderabad-to...,AdSri Krishna Travels,,19:30,11h 20m,06:50,3.6,INR 1500,31 Seats available
